### Objectives

- Launch the trained YOLO model (models/yolov8n_handwritten_20241209.pt) on the images in the data folder. 
- Create 1 XML file for each image 
- Save the results in the results folder. 

INPUTS: 
- data/FOLDERS/
- models/yolov8n_handwritten_20241209.pt

OUTPUTS: 
- data/FOLDERS/json
- data/FOLDERS/xml





In [7]:
#activate the virtual environment

!.\venv\Scripts\activate


In [2]:
import os

os.chdir('D:/GitHub/trocr_handwritten')
PATH_REPO=os.getcwd()
PATH_DATA = os.path.join(PATH_REPO,'data/20241106_yolo/train')
PATH_MODEL = os.path.join(PATH_REPO,'models')
#.py functions to parse page using model
PATH_PARSE = os.path.join(PATH_REPO,'trocr_handwritten/parse')

parse_function=os.path.join(PATH_PARSE,'layout_ft_yolo.py')
model=os.path.join(PATH_MODEL,'yolov8n_handwritten_20241209.pt')
PATH_PARSE

'D:\\GitHub\\trocr_handwritten\\trocr_handwritten/parse'

In [9]:
import argparse
from pathlib import Path
import json
from os.path import join
from doclayout_yolo import YOLOv10

ModuleNotFoundError: No module named 'huggingface_hub'

In [3]:

python trocr_handwritten/parse/layout_ft_yolo.py --path_folder data/20241106_yolo/train --path_model models/yolov8n_handwritten_20241209.pt --conf 0.2 --iou 0.5

Traceback (most recent call last):
  File "D:\GitHub\trocr_handwritten\trocr_handwritten\parse\layout_ft_yolo.py", line 7, in <module>
    from doclayout_yolo import YOLOv10
  File "D:\GitHub\trocr_handwritten\layout\DocLayout-YOLO\doclayout_yolo\__init__.py", line 5, in <module>
    from doclayout_yolo.data.explorer.explorer import Explorer
  File "D:\GitHub\trocr_handwritten\layout\DocLayout-YOLO\doclayout_yolo\data\explorer\explorer.py", line 18, in <module>
    from doclayout_yolo.models.yolo.model import YOLO
  File "D:\GitHub\trocr_handwritten\layout\DocLayout-YOLO\doclayout_yolo\models\__init__.py", line 6, in <module>
    from .yolov10 import YOLOv10
  File "D:\GitHub\trocr_handwritten\layout\DocLayout-YOLO\doclayout_yolo\models\yolov10\__init__.py", line 1, in <module>
    from .model import YOLOv10
  File "D:\GitHub\trocr_handwritten\layout\DocLayout-YOLO\doclayout_yolo\models\yolov10\model.py", line 7, in <module>
    from huggingface_hub import PyTorchModelHubMixin
ModuleNo